In [1]:
import sys
from pathlib import Path
import warnings

warnings.filterwarnings("ignore", module="IPython")


def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False


def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book


def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip subdirectories from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ("src",):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ("airquality",):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ("notebooks",):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir)
    print("Local environment")

print(f"Root dir: {root_dir}")

# Add the root directory to the `PYTHONPATH`
if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

# Set the environment variables from the file <root_dir>/.env
from src import config

settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Root dir: /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn
Added the following directory to the PYTHONPATH: /Users/max/Repos/KTH/pm25-forecast-openmeteo-aqicn
HopsworksSettings initialized!


In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from src.airquality import util
from src import config
import json
import os
import warnings
import pandas as pd

warnings.filterwarnings("ignore")

# Setup

In [3]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store()
secrets = hopsworks.get_secrets_api()
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value

# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name="air_quality_all",
    version=1,
)
weather_fg = fs.get_feature_group(
    name="weather_all",
    version=1,
)

# Optional single sensor mode if ID is set in .env
sensor_id_param = getattr(settings, 'SENSOR_ID', None)

if sensor_id_param:
    # Read one secret for single sensor mode
    secret_name = f"SENSOR_LOCATION_JSON_{sensor_id_param}"
    location_str = secrets.get_secret(secret_name).value
    locations = {sensor_id_param: json.loads(location_str)}
else:
    # Read all individual secrets in batch mode
    all_secrets = secrets.get_secrets()
    locations = {}
    for secret in all_secrets:
        if secret.name.startswith("SENSOR_LOCATION_JSON_"):
            sensor_id = secret.name.replace("SENSOR_LOCATION_JSON_", "")
            location_str = secrets.get_secret(secret.name).value
            if location_str:
                locations[sensor_id] = json.loads(location_str)

today = datetime.date.today()

2025-11-16 15:29:19,724 INFO: Initializing external client
2025-11-16 15:29:19,725 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-16 15:29:21,424 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279179


# Methods

In [4]:
def add_rolling_window_feature(df, window_days=3, column="pm25", new_column="pm25_rolling_3d"):
    df = df.sort_values(["sensor_id", "date"]).copy()
    
    df_indexed = df.set_index("date", append=False)
    df_indexed[f"{column}_shifted"] = df_indexed.groupby("sensor_id")[column].shift(1)
    
    df[new_column] = (
        df_indexed.groupby("sensor_id")[f"{column}_shifted"]
        .rolling(window=f"{window_days}D", min_periods=1)
        .mean()
        .reset_index(level=0, drop=True)
        .values
    )

    return df


def add_lagged_features(df, column="pm25", lags=[1, 2, 3]):
    df = df.sort_values(["sensor_id", "date"]).copy()
    
    for lag in lags:
        new_column = f"{column}_lag_{lag}d"
        df[new_column] = df.groupby("sensor_id")[column].shift(lag)
    
    return df

In [5]:
def get_daily_weather_forecast(city, latitude, longitude):
    hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
    hourly_df = hourly_df.set_index("date")
    daily_df = hourly_df.between_time("11:59", "12:01")
    daily_df = daily_df.reset_index()
    daily_df["date"] = pd.to_datetime(daily_df["date"]).dt.date
    daily_df["date"] = pd.to_datetime(daily_df["date"])
    daily_df["city"] = city

    return daily_df


def fetch_data_for_location(location):
    country = location["country"]
    city = location["city"]
    street = location["street"]
    aqicn_url = location["aqicn_url"]
    latitude = location["latitude"]
    longitude = location["longitude"]

    aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
    daily_df = get_daily_weather_forecast(city, latitude, longitude)
    return aq_today_df, daily_df

# Script

In [6]:
aqs = []
weathers = []
for sensor, location in locations.items():
    aq_today_df, weather_daily_forecast_df = fetch_data_for_location(location)

    # Air quality FG shape
    aq_today_df = aq_today_df.assign(
        sensor_id=str(sensor),
        street=location["street"],
        city=location["city"],
        country=location["country"],
        feed_url=location["aqicn_url"],
    )
    aq_today_df["date"] = pd.to_datetime(aq_today_df["date"])

    # Weather FG shape
    weather_daily_forecast_df = weather_daily_forecast_df.assign(
        sensor_id=str(sensor),
        city=location["city"],
        latitude=location["latitude"],
        longitude=location["longitude"],
    )
    weather_daily_forecast_df["date"] = pd.to_datetime(
        weather_daily_forecast_df["date"]
    )

    aqs.append(aq_today_df)
    weathers.append(weather_daily_forecast_df)

Coordinates 57.75°N 12.0°E
Elevation 72.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 57.75°N 12.0°E
Elevation 21.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 57.75°N 12.0°E
Elevation 21.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 57.5°N 12.0°E
Elevation 55.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 57.75°N 12.0°E
Elevation 27.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 57.75°N 12.0°E
Elevation 24.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 57.75°N 12.0°E
Elevation 15.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 57.75°N 12.0°E
Elevation 12.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 57.75°N 11.75°E
Elevation 7.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 57.75°N 12.0°E
Elevation 76.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
C

In [7]:
aq_df = pd.concat(aqs)
aq_df["pm25"] = pd.to_numeric(aq_df["pm25"], errors="coerce").astype("float64")
aq_df["date"] = pd.to_datetime(aq_df["date"])
aq_df = aq_df.drop(columns=["url"], errors="ignore")

# Get historical data (last 4 days) for rolling window and lagged features
historical_start = today - datetime.timedelta(days=4)
historical_df = pd.DataFrame()
try:
    historical_df = air_quality_fg.read()
    if not historical_df.empty:
        historical_df["date"] = pd.to_datetime(historical_df["date"]).dt.date
        historical_df = historical_df[
            (historical_df["date"] >= historical_start) & (historical_df["date"] < today)
        ][["date", "sensor_id", "pm25"]]
        historical_df["date"] = pd.to_datetime(historical_df["date"])
except Exception:
    pass

# Combine historical + new data and calculate rolling window
combined_df = pd.concat([historical_df, aq_df], ignore_index=True) if not historical_df.empty else aq_df
combined_df = add_rolling_window_feature(combined_df, window_days=3, column="pm25", new_column="pm25_rolling_3d")
combined_df = add_lagged_features(combined_df, column="pm25", lags=[1, 2, 3])
aq_df = combined_df[combined_df["date"].dt.date == today].copy()

air_quality_fg.insert(aq_df)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.95s) 
2025-11-16 15:29:54,108 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279179/fs/1265797/fg/1721733


Uploading Dataframe: 100.00% |██████████| Rows 15/15 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_all_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279179/jobs/named/air_quality_all_1_offline_fg_materialization/executions


(Job('air_quality_all_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 761903
         }
       },
       "result": {
         "observed_value": 1.0,
         "element_count": 15,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-16T02:29:54.000108Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expe

In [8]:
weather_df = pd.concat(weathers)
weather_df["date"] = pd.to_datetime(weather_df["date"])
weather_fg.insert(weather_df)

2025-11-16 15:30:08,423 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279179/fs/1265797/fg/1721734


Uploading Dataframe: 100.00% |██████████| Rows 105/105 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_all_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279179/jobs/named/weather_all_1_offline_fg_materialization/executions


(Job('weather_all_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 761904
         }
       },
       "result": {
         "observed_value": 2.545584201812744,
         "element_count": 105,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-16T02:30:08.000423Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_t